In [ ]:
# ============================================================================
# MAG DATA DOWNLOAD DIAGNOSTIC - FIXED VERSION
# Run this entire cell and share ALL output
# ============================================================================

import sys
import os
import glob
from datetime import datetime

print("="*80)
print("MAG DATA DOWNLOAD DIAGNOSTIC (FIXED)")
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

# ============================================================================
# SECTION 1: IMPORT & SETUP
# ============================================================================
print("\n" + "="*80)
print("1. IMPORT & SETUP")
print("="*80)

try:
    import plotbot
    from plotbot import config, mag_rtn_4sa
    # FIX: Import the case-insensitive lookup function!
    from plotbot.data_classes.data_types import data_types, get_local_path, get_data_type_config
    from plotbot.data_download_pyspedas import download_spdf_data
    import numpy as np
    print("✅ Plotbot imported successfully")
    print("✅ get_data_type_config imported (case-insensitive lookup)")
except Exception as e:
    print(f"❌ Failed to import plotbot: {e}")
    import traceback
    traceback.print_exc()
    raise SystemExit("Cannot continue without plotbot")

# Test parameters
trange = ['2020-01-29/18:00:00', '2020-01-29/20:00:00']
data_type = 'mag_rtn_4sa'

print(f"\nTest time range: {trange}")
print(f"Data type: {data_type}")

# ============================================================================
# SECTION 2: MAG CONFIGURATION (FIXED!)
# ============================================================================
print("\n" + "="*80)
print("2. MAG CONFIGURATION (using case-insensitive lookup)")
print("="*80)

try:
    # FIX: Use case-insensitive lookup!
    mag_config = get_data_type_config(data_type)
    
    if not mag_config:
        print(f"❌ CRITICAL: No configuration found for '{data_type}' in data_types!")
        print(f"Available data types: {list(data_types.keys())[:10]}...")
        print(f"Note: Used case-insensitive lookup via get_data_type_config()")
    else:
        print(f"✅ MAG configuration found (case-insensitive lookup worked!)")
        print(f"\nConfiguration details:")
        for key in ['data_level', 'local_path', 'file_pattern_import', 'pyspedas_name', 'pyspedas_datatype']:
            value = mag_config.get(key, 'NOT SET')
            print(f"  {key:25s}: {value}")
        
        # Check data sources
        data_sources = mag_config.get('data_sources', [])
        print(f"\n  data_sources: {data_sources}")
        if 'spdf' in data_sources:
            print(f"    ✅ SPDF download enabled")
        if 'berkeley' in data_sources:
            print(f"    ✅ Berkeley fallback enabled")
            
except Exception as e:
    print(f"❌ Error reading MAG config: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 3: PATH ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("3. PATH ANALYSIS")
print("="*80)

try:
    print(f"\nPlotbot config:")
    print(f"  config.data_dir:    {config.data_dir}")
    print(f"  Exists:             {os.path.exists(config.data_dir)}")
    
    if mag_config:
        print(f"\nExpected MAG paths:")
        
        # Get the local path plotbot expects
        local_path = get_local_path(data_type)
        print(f"  get_local_path():   {local_path}")
        if local_path:
            print(f"  Exists:             {os.path.exists(local_path)}")
            
            # Create the path if it doesn't exist
            if not os.path.exists(local_path):
                try:
                    os.makedirs(local_path, exist_ok=True)
                    print(f"  → Created directory: {local_path}")
                except:
                    pass
            
            # Check for 2020 subdirectory
            year_path = os.path.join(local_path, '2020')
            print(f"\n  2020 subdirectory:  {year_path}")
            print(f"  Exists:             {os.path.exists(year_path)}")
    else:
        print("  Skipping path analysis - no config found")
    
    # Check PySpedas default paths
    print(f"\nPySpedas paths to check:")
    pyspedas_paths = [
        os.path.join(config.data_dir, 'psp_data/fields/l2/mag_rtn_4_per_cycle'),
        './psp_data/fields/l2/mag_rtn_4_per_cycle',
        os.path.join(os.getcwd(), 'psp_data/fields/l2/mag_rtn_4_per_cycle')
    ]
    
    for path in pyspedas_paths:
        print(f"  {path}")
        print(f"    Exists: {os.path.exists(path)}")
        if os.path.exists(path):
            year_dir = os.path.join(path, '2020')
            if os.path.exists(year_dir):
                files = glob.glob(os.path.join(year_dir, '*20200129*.cdf'))
                print(f"    Files for 2020-01-29: {len(files)}")
                for f in files:
                    print(f"      {os.path.basename(f)}")
                    
except Exception as e:
    print(f"❌ Error in path analysis: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 4: TEST PLOTBOT WORKFLOW
# ============================================================================
print("\n" + "="*80)
print("4. TEST PLOTBOT WORKFLOW")
print("="*80)

try:
    print(f"\nCalling plotbot(trange, mag_rtn_4sa.br)...")
    from plotbot import plotbot as plotbot_func
    
    # Call plotbot to load data
    plotbot_func(trange, mag_rtn_4sa.br)
    
    # Check if data loaded
    br_data = mag_rtn_4sa.br.data
    br_datetime = mag_rtn_4sa.br.datetime_array
    
    print(f"\nResults:")
    print(f"  br.data type:           {type(br_data)}")
    print(f"  br.data shape:          {getattr(br_data, 'shape', 'N/A')}")
    print(f"  datetime_array type:    {type(br_datetime)}")
    print(f"  datetime_array shape:   {getattr(br_datetime, 'shape', 'N/A')}")
    
    # Check for valid data
    if hasattr(br_data, '__len__') and len(br_data) > 0:
        print(f"  ✅ Data loaded! {len(br_data)} points")
        print(f"  First datetime: {br_datetime[0] if len(br_datetime) > 0 else 'N/A'}")
        print(f"  Last datetime:  {br_datetime[-1] if len(br_datetime) > 0 else 'N/A'}")
    else:
        print(f"  ❌ No data loaded!")
        
except Exception as e:
    print(f"❌ Error during plotbot workflow: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 5: FILE SEARCH
# ============================================================================
print("\n" + "="*80)
print("5. COMPREHENSIVE FILE SEARCH")
print("="*80)

print(f"\nSearching for MAG files with '20200129' in name...\n")

search_dirs = [
    (config.data_dir, "Plotbot data dir"),
    (os.getcwd(), "Current working directory"),
]

for search_dir, label in search_dirs:
    try:
        print(f"Searching in: {search_dir} ({label})")
        mag_files = glob.glob(os.path.join(search_dir, '**/*20200129*.cdf'), recursive=True)
        if mag_files:
            print(f"  ✅ Found {len(mag_files)} MAG file(s):")
            for f in mag_files:
                print(f"    - {f}")
        else:
            print(f"  ❌ No MAG files found")
    except Exception as e:
        print(f"  ❌ Error searching: {e}")
    print()

# ============================================================================
# SECTION 6: DIAGNOSIS SUMMARY
# ============================================================================
print("\n" + "="*80)
print("6. DIAGNOSIS SUMMARY")
print("="*80)

if mag_config:
    print(f"\n✅ Case-insensitive config lookup: WORKING")
    print(f"✅ MAG configuration found for '{data_type}'")
else:
    print(f"\n❌ Case-insensitive config lookup: FAILED")
    print(f"❌ This is the root cause of the issue!")

print("\n" + "="*80)
print("END OF DIAGNOSTIC")
print("="*80)
